# Processing of Slocum Glider-AD2CP Data: RU29 2020

jgradone@marine.rutgers.edu     03/10/2022    Initial

**This Jupyter Notebook is intended to:**<br>
1) Read glider data frome ERDDAP <br>
2) Read in AD2CP data processed to NetCDFs using the Nortek MIDAS software as an xarray DataSet and combine all into one massive DataSet <br>

**Then in a big for loop:** <br>
3) Pull out just one glider segment (based on a unique file name) and the corresponding dead-reckoned depth average current. Use the start and end times to subset the AD2CP data <br>
4) Correct for local speed of sound <br>
5) QAQC AD2CP data <br>
6) Correct beams to level true-depth <br>
7) Bin map AD2CP data <br>
8) AD2CP Coordinate transformation from beam to XYZ to ENU <br>
9) Least squares linear inversion on ADCP velocities referenced to true ocean velocity through a depth averaged urrent constraint <br>
10) Save output from each segment<br>

*Details/comments on what the functions are actually doing in the source code*

In [1]:
# Imports
import scipy.interpolate as interp
from scipy.sparse.linalg import lsqr
import scipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import netCDF4 as nc
import math
import datetime
import xarray as xr
import matplotlib.dates as mdates
import dask.array as da
from erddapy import ERDDAP
from netCDF4 import Dataset
import gsw
import cmocean.cm as cmo
import sys
from datetime import datetime


## To import functions from Slocum-AD2CP GitHub repository, make this path the path to where the repo exists locally
sys.path.insert(0,'/home/jg1200/GitHub/Slocum-AD2CP/src/analysis/')
sys.path.insert(0,'/home/jg1200/GitHub/Slocum-AD2CP/src/data/')
from make_dataset import correct_sound_speed, beam_true_depth, cell_vert, binmap_adcp, beam2enu, inversion_new, inversion, qaqc_pre_coord_transform, qaqc_post_coord_transform, shear_method
from analysis import get_erddap_dataset



## Step 1: Load glider data

In [2]:
ds_id = 'ru29-20200908T1623-trajectory-raw-delayed'

## Load flight data
variables = ['depth', 'latitude', 'longitude', 'time', 'sci_water_temp', 'sci_water_cond','source_file', 'm_water_vx', 'm_water_vy', 'm_heading']
gdf = get_erddap_dataset(ds_id, server='http://slocum-data.marine.rutgers.edu/erddap', variables = variables, filetype='dataframe')
gdf.columns = variables

## Great way to find start and end times!!
start_times = gdf.groupby('source_file').first().time.values
end_times   = gdf.groupby('source_file').last().time.values
## Remove time zone for slicing ad2cp times
start_times2 = pd.to_datetime(start_times).tz_localize(None)
end_times2 = pd.to_datetime(end_times).tz_localize(None)

gdf

,depth,latitude,longitude,time,sci_water_temp,sci_water_cond,source_file,m_water_vx,m_water_vy,m_heading
0,0.00000,18.175290,-64.802942,2020-09-08T16:23:40.15Z,0.0000,0.00000,ru29-2020-251-4-0-mbd(04270000),NaN,NaN,NaN
1,0.00000,18.175290,-64.802942,2020-09-08T16:23:47.43Z,0.0000,0.00000,ru29-2020-251-4-0-mbd(04270000),NaN,NaN,NaN
2,NaN,18.175290,-64.802942,2020-09-08T16:23:47.56Z,NaN,NaN,ru29-2020-251-4-0-mbd(04270000),NaN,NaN,1.80293
3,NaN,18.175290,-64.802942,2020-09-08T16:24:48.43Z,NaN,NaN,ru29-2020-251-4-0-mbd(04270000),NaN,NaN,1.80293
4,NaN,18.176045,-64.804093,2020-09-08T16:25:14.45Z,NaN,NaN,ru29-2020-251-4-0-mbd(04270000),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5087216,0.18886,18.139090,-64.784373,2020-11-12T07:31:32.76Z,27.6073,5.43973,ru29-2020-315-0-9-mbd(04470009),NaN,NaN,NaN
5087217,0.18886,18.139090,-64.784373,2020-11-12T07:31:34.77Z,27.6072,5.43980,ru29-2020-315-0-9-mbd(04470009),NaN,NaN,NaN
5087218,0.14910,18.139090,-64.784373,2020-11-12T07:31:36.79Z,27.6093,5.43990,ru29-2020-315-0-9-mbd(04470009),NaN,NaN,NaN
5087219,0.15904,18.139090,-64.784373,2020-11-12T07:31:38.80Z,27.6093,5.43993,ru29-2020-315-0-9-mbd(04470009),NaN,NaN,NaN


## Step 2: Load in AD2CP data frame

In [3]:
path = '/home/jg1200/Data/GliderData/RU29/RU29_2020_AD2CP_NC/'
files = np.sort(glob.glob(path+'*.nc'))

tot_ad2cp = xr.open_mfdataset(files,group='Data/Burst/', concat_dim="time", combine="nested")
## I think this is a documented bug in the xarray open_mfdataset function for grouped NetCDFs
config = xr.open_dataset(files[0],group='Config')
## So, just assigning the global attributes from the first file to the combined file
tot_ad2cp = tot_ad2cp.assign_attrs(config.attrs)
tot_ad2cp = tot_ad2cp.rename({'Velocity Range':'VelocityRange','Correlation Range':'CorrelationRange','Amplitude Range':'AmplitudeRange'})
# 2 House-keeping steps
# 1) Roll shifted 180 for some reason
tot_ad2cp['Roll'] = tot_ad2cp['Roll']-180
# 2) Surface depth is 10 meters. Needed for at least for the 2020 and both 2021 deployments, per conversations with Sven from Nortek.
tot_ad2cp['Pressure'] = tot_ad2cp['Pressure']-10
# Put time on x-dimension
tot_ad2cp = tot_ad2cp.transpose()
tot_ad2cp

<xarray.Dataset>
Dimensions:                    (time: 11580371, VelocityRange: 40, CorrelationRange: 40, AmplitudeRange: 40, Physicalbeam_dim: 4, AHRSRotationMatrix_dim: 9)
Coordinates:
  * time                       (time) datetime64[ns] 2020-09-08T17:52:48.0320...
  * VelocityRange              (VelocityRange) float32 0.7 1.2 1.7 ... 19.7 20.2
  * CorrelationRange           (CorrelationRange) float32 0.7 1.2 ... 19.7 20.2
  * AmplitudeRange             (AmplitudeRange) float32 0.7 1.2 ... 19.7 20.2
Dimensions without coordinates: Physicalbeam_dim, AHRSRotationMatrix_dim
Data variables: (12/52)
    VelocityBeam1              (VelocityRange, time) float32 dask.array<chunksize=(40, 150080), meta=np.ndarray>
    VelocityBeam2              (VelocityRange, time) float32 dask.array<chunksize=(40, 150080), meta=np.ndarray>
    VelocityBeam3              (VelocityRange, time) float32 dask.array<chunksize=(40, 150080), meta=np.ndarray>
    VelocityBeam4              (VelocityRange, time) float32 dask.array<chunksize=(40, 150080), meta=np.ndarray>
    CorrelationBeam1           (CorrelationRange, time) float32 dask.array<chunksize=(40, 150080), meta=np.ndarray>
    CorrelationBeam2           (CorrelationRange, time) float32 dask.array<chunksize=(40, 150080), meta=np.ndarray>
    ...                         ...
    QuaternionX                (time) float32 dask.array<chunksize=(150080,), meta=np.ndarray>
    QuaternionY                (time) float32 dask.array<chunksize=(150080,), meta=np.ndarray>
    QuaternionZ                (time) float32 dask.array<chunksize=(150080,), meta=np.ndarray>
    AHRSGyroX                  (time) float32 dask.array<chunksize=(150080,), meta=np.ndarray>
    AHRSGyroY                  (time) float32 dask.array<chunksize=(150080,), meta=np.ndarray>
    AHRSGyroZ                  (time) float32 dask.array<chunksize=(150080,), meta=np.ndarray>
Attributes: (12/350)
    exportRevision:                               4421
    File_ins_runTime:                             377235750.0
    File_ins_runTime_description:                 Length of time data was col...
    File_ins_name:                                AD2CPRaw
    File_ins_name_description:                    Name of instrument
    File_ins_typeName:                            
    ...                                           ...
    instrumentOffsetTime:                         0.0
    instrumentOffsetTime_description:             Offset from instrument time...
    timeZoneOffset:                               -14400
    timeZoneOffset_description:                   Offset of platform local ti...
    instrumentOffsetTimeMatlab:                   0.0
    instrumentOffsetTimeMatlab_description:       Offset from instrument time...

## Steps 3-10: Big loop to process velocity data and save output

In [5]:
# #for x in np.arange(865,len(start_times)):

# # AD2CP only on starting on segment 408 and off after segment 976
# for x in np.arange(408,976):

#     ## Subset glider df to one segment
#     subsetgdf = gdf[(gdf.time >= start_times[x]) & (gdf.time <= end_times[x])]
#     ## Pull out last non-NaN DAC
#     ind         = np.argwhere(np.isnan(subsetgdf.m_water_vx).ravel()==False).flatten()
    
#     ## Check if there is a DAC, if not continue onto next segment.
#     if len(ind) > 0:
#         vx          = subsetgdf.m_water_vx.iloc[ind[-1]]
#         vy          = subsetgdf.m_water_vy.iloc[ind[-1]]
        
#         ## Pull out last non-NaN lat/lon
#         ind1         = np.argwhere(np.isnan(subsetgdf.longitude).ravel()==False).flatten()   
#         vx_start_lon = subsetgdf.longitude.iloc[ind1[0]]
#         vx_start_lat = subsetgdf.latitude.iloc[ind1[0]]
#         vx_end_lon   = subsetgdf.longitude.iloc[ind1[-1]]
#         vx_end_lat   = subsetgdf.latitude.iloc[ind1[-1]]
#         ## Not using the index here because I want the segment start and end times
#         vx_start_tm  = subsetgdf.time.iloc[0]
#         vx_end_tm    = subsetgdf.time.iloc[-1]

#         ## Check is glider has a max depth greater than 10 meters. If not, continue to next segment.
#         ## This is needed because the glider opens and closes a log file at the surface while sending data and
#         ## so indexing by unique file names gives a segment with just surface data (which we don't want).
#         if subsetgdf.depth.max() < 10:
#             continue

#         ## Subset AD2CP dataframe based on segment start and end times
#         ad2cp_time_ind = np.where((tot_ad2cp.time >= start_times2[x]) & (tot_ad2cp.time <= end_times2[x]))[0]
#         subset_ad2cp = tot_ad2cp.sel(time=tot_ad2cp.time[ad2cp_time_ind])

#         ## Just check if there is still data after the subsetting 
#         if len(subset_ad2cp.time) > 0:
#             ## Now process AD2CP!

#             ## Correct for local speed of sound
#             subset_ad2cp = correct_sound_speed(subset_ad2cp)
#             now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
#             print("Finished Sound Speed Correction", x ,"out of",len(start_times),"at" ,now)

#             # QAQC pre-coordinate transformation
#             subset_ad2cp = qaqc_pre_coord_transform(subset_ad2cp, corr_threshold = 50, max_amplitude = 75)
#             now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
#             print("Finished QAQC", x ,"out of",len(start_times),"at" ,now)

#             ## Correct beams to level true-depth
#             subset_ad2cp = beam_true_depth(subset_ad2cp)
#             now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
#             print("Finished Beam to True Depth", x ,"out of",len(start_times),"at" ,now)

#             ## Bin map
#             subset_ad2cp = binmap_adcp(subset_ad2cp)
#             now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
#             print("Finished Bin Mapping", x ,"out of",len(start_times),"at" ,now)

#             ## Beam to ENU
#             subset_ad2cp = beam2enu(subset_ad2cp)
#             now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
#             print("Finished Beam to ENU", x ,"out of",len(start_times),"at" ,now)
            
#             ## QAQC post-coordinate transformation
#             subset_ad2cp = qaqc_post_coord_transform(subset_ad2cp, high_velocity_threshold=0.75, surface_depth_to_filter = 5)
#             now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
#             print("Finished QAQC post-coordinate transformation", x ,"out of",len(start_times),"at" ,now)
            
#             # ## Now ready for inversion!
#             #O_ls, G_ls, bin_new = inversion(U=subset_ad2cp.UVelocity.transpose().values,V=subset_ad2cp.VVelocity.transpose().values,H=1000,dz=5,u_daverage=vx,v_daverage=vy,bins = subset_ad2cp['VelocityRange'], depth = subset_ad2cp['Pressure'])
#             O_ls, G_ls, bin_new,obs_per_bin  = inversion_new(U=subset_ad2cp.UVelocity.values,V=subset_ad2cp.VVelocity.values,dz=10,u_daverage=vx,v_daverage=vy,bins = subset_ad2cp['VelocityRange'].values, depth = subset_ad2cp['Pressure'].values,wDAC=5, wSmoothness=0.02)
#             now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
#             print("Finished Inversion", x ,"out of",len(start_times),"at" ,now)

#             ###############################################
#             #             Save master dataset             #
#             ###############################################
#             fname = "/home/jg1200/Data/GliderData/RU29/RU29_2020_AD2CP_NC/Processed/RU29_2020_AD2CP_Processed_Segment_{}.csv".format(x)
#             ## Make into a dataframe to save as a CSV
#             d = {'inversion_u': np.real(O_ls), 'inversion_v': np.imag(O_ls), "inversion_depth": bin_new,
#                  "start_lon": np.tile(vx_start_lon,len(bin_new)), "start_lat": np.tile(vx_start_lat,len(bin_new)),
#                  "end_lon": np.tile(vx_end_lon,len(bin_new)), "end_lat": np.tile(vx_end_lat,len(bin_new)),
#                  "start_tm": np.tile(vx_start_tm, len(bin_new)), "end_tm": np.tile(vx_end_tm, len(bin_new)),
#                  "obs_per_bin": obs_per_bin}

            
#             df = pd.DataFrame(data=d)
#             df.to_csv(fname) 
#             now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
#             print("Finished Writing Data", x ,"out of",len(start_times),"at" ,now)
#             del subset_ad2cp
#         else:
#             del subset_ad2cp

            

## Do the same thing but for the shear method now

In [ ]:
#for x in np.arange(865,len(start_times)):

# AD2CP only on starting on segment 408 and off after segment 976
for x in np.arange(408,976):

    ## Subset glider df to one segment
    subsetgdf = gdf[(gdf.time >= start_times[x]) & (gdf.time <= end_times[x])]
    ## Pull out last non-NaN DAC
    ind         = np.argwhere(np.isnan(subsetgdf.m_water_vx).ravel()==False).flatten()
    
    ## Check if there is a DAC, if not continue onto next segment.
    if len(ind) > 0:
        vx          = subsetgdf.m_water_vx.iloc[ind[-1]]
        vy          = subsetgdf.m_water_vy.iloc[ind[-1]]
        
        ## Pull out last non-NaN lat/lon
        ind1         = np.argwhere(np.isnan(subsetgdf.longitude).ravel()==False).flatten()   
        vx_start_lon = subsetgdf.longitude.iloc[ind1[0]]
        vx_start_lat = subsetgdf.latitude.iloc[ind1[0]]
        vx_end_lon   = subsetgdf.longitude.iloc[ind1[-1]]
        vx_end_lat   = subsetgdf.latitude.iloc[ind1[-1]]
        ## Not using the index here because I want the segment start and end times
        vx_start_tm  = subsetgdf.time.iloc[0]
        vx_end_tm    = subsetgdf.time.iloc[-1]

        ## Check is glider has a max depth greater than 10 meters. If not, continue to next segment.
        ## This is needed because the glider opens and closes a log file at the surface while sending data and
        ## so indexing by unique file names gives a segment with just surface data (which we don't want).
        if subsetgdf.depth.max() < 10:
            continue

        ## Subset AD2CP dataframe based on segment start and end times
        ad2cp_time_ind = np.where((tot_ad2cp.time >= start_times2[x]) & (tot_ad2cp.time <= end_times2[x]))[0]
        subset_ad2cp = tot_ad2cp.sel(time=tot_ad2cp.time[ad2cp_time_ind])

        ## Just check if there is still data after the subsetting 
        if len(subset_ad2cp.time) > 0:
            ## Now process AD2CP!

            ## Correct for local speed of sound
            subset_ad2cp = correct_sound_speed(subset_ad2cp)
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Sound Speed Correction", x ,"out of",len(start_times),"at" ,now)

            # QAQC pre-coordinate transformation
            subset_ad2cp = qaqc_pre_coord_transform(subset_ad2cp, corr_threshold = 50, max_amplitude = 75)
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished QAQC", x ,"out of",len(start_times),"at" ,now)

            ## Correct beams to level true-depth
            subset_ad2cp = beam_true_depth(subset_ad2cp)
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Beam to True Depth", x ,"out of",len(start_times),"at" ,now)

            ## Bin map
            subset_ad2cp = binmap_adcp(subset_ad2cp)
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Bin Mapping", x ,"out of",len(start_times),"at" ,now)

            ## Beam to ENU
            subset_ad2cp = beam2enu(subset_ad2cp)
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Beam to ENU", x ,"out of",len(start_times),"at" ,now)
            
            ## QAQC post-coordinate transformation
            subset_ad2cp = qaqc_post_coord_transform(subset_ad2cp, high_velocity_threshold=0.75, surface_depth_to_filter = 5)
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished QAQC post-coordinate transformation", x ,"out of",len(start_times),"at" ,now)
            
            ## Now ready for shear method!!            
            dz=10
            U_shear_method, V_shear_method, U_prof_std, V_prof_std, bin_new_shear, obs_per_bin_shear = shear_method(U=subset_ad2cp.UVelocity.values,V=subset_ad2cp.UVelocity.values,dz=dz,vx=vx,vy=vy,bins = subset_ad2cp['VelocityRange'].values, depth = subset_ad2cp['Pressure'].values)
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Shear Method", x ,"out of",len(start_times),"at" ,now)

            ###############################################
            #             Save master dataset             #
            ###############################################
            fname = "/home/jg1200/Data/GliderData/RU29/RU29_2020_AD2CP_NC/Processed_shear_{}_meter_vertical_resoultion/RU29_2020_AD2CP_Processed_Segment_{}.csv".format(dz,x)
            ## Make into a dataframe to save as a CSV
            
            # Make into a dataframe to save as a CSV
            d = {'U_shear_method': U_shear_method, 'V_shear_method': V_shear_method, "bin_new_shear": bin_new_shear,
                 "U_prof_std": U_prof_std, "V_prof_std": V_prof_std, "obs_per_bin_shear": obs_per_bin_shear,
                 "start_lon": np.tile(vx_start_lon,len(bin_new_shear)), "start_lat": np.tile(vx_start_lat,len(bin_new_shear)),
                 "end_lon": np.tile(vx_end_lon,len(bin_new_shear)), "end_lat": np.tile(vx_end_lat,len(bin_new_shear)),
                 "start_tm": np.tile(vx_start_tm, len(bin_new_shear)), "end_tm": np.tile(vx_end_tm, len(bin_new_shear))}


            
            df = pd.DataFrame(data=d)
            df.to_csv(fname) 
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Writing Data", x ,"out of",len(start_times),"at" ,now)
            del subset_ad2cp
        else:
            del subset_ad2cp

            

Finished Sound Speed Correction 408 out of 1081 at 06/22/22 11:17:21
Finished QAQC 408 out of 1081 at 06/22/22 11:17:23
Finished Beam to True Depth 408 out of 1081 at 06/22/22 11:17:25
Finished Bin Mapping 408 out of 1081 at 06/22/22 11:17:25
Finished Beam to ENU 408 out of 1081 at 06/22/22 11:19:59
Finished QAQC post-coordinate transformation 408 out of 1081 at 06/22/22 11:19:59
Finished Shear Method 408 out of 1081 at 06/22/22 11:20:13
Finished Writing Data 408 out of 1081 at 06/22/22 11:20:13
Finished Sound Speed Correction 410 out of 1081 at 06/22/22 11:20:15
Finished QAQC 410 out of 1081 at 06/22/22 11:20:16
Finished Beam to True Depth 410 out of 1081 at 06/22/22 11:20:18
Finished Bin Mapping 410 out of 1081 at 06/22/22 11:20:19
Finished Beam to ENU 410 out of 1081 at 06/22/22 11:23:02
Finished QAQC post-coordinate transformation 410 out of 1081 at 06/22/22 11:23:02
Finished Shear Method 410 out of 1081 at 06/22/22 11:23:25
Finished Writing Data 410 out of 1081 at 06/22/22 11:23:2

Finished Bin Mapping 440 out of 1081 at 06/22/22 12:08:31
Finished Beam to ENU 440 out of 1081 at 06/22/22 12:11:03
Finished QAQC post-coordinate transformation 440 out of 1081 at 06/22/22 12:11:03
Finished Shear Method 440 out of 1081 at 06/22/22 12:11:20
Finished Writing Data 440 out of 1081 at 06/22/22 12:11:49
Finished Sound Speed Correction 443 out of 1081 at 06/22/22 12:11:52
Finished QAQC 443 out of 1081 at 06/22/22 12:11:52
Finished Beam to True Depth 443 out of 1081 at 06/22/22 12:11:54
Finished Bin Mapping 443 out of 1081 at 06/22/22 12:11:54
Finished Beam to ENU 443 out of 1081 at 06/22/22 12:14:26
Finished QAQC post-coordinate transformation 443 out of 1081 at 06/22/22 12:14:26
Finished Shear Method 443 out of 1081 at 06/22/22 12:14:43
Finished Writing Data 443 out of 1081 at 06/22/22 12:14:43
Finished Sound Speed Correction 445 out of 1081 at 06/22/22 12:14:45
Finished QAQC 445 out of 1081 at 06/22/22 12:14:46
Finished Beam to True Depth 445 out of 1081 at 06/22/22 12:14:4

Finished Shear Method 473 out of 1081 at 06/22/22 12:58:33
Finished Writing Data 473 out of 1081 at 06/22/22 12:58:33
Finished Sound Speed Correction 474 out of 1081 at 06/22/22 12:58:35
Finished QAQC 474 out of 1081 at 06/22/22 12:58:36
Finished Beam to True Depth 474 out of 1081 at 06/22/22 12:58:38
Finished Bin Mapping 474 out of 1081 at 06/22/22 12:58:39
Finished Beam to ENU 474 out of 1081 at 06/22/22 13:01:09
Finished QAQC post-coordinate transformation 474 out of 1081 at 06/22/22 13:01:09
Finished Shear Method 474 out of 1081 at 06/22/22 13:01:24
Finished Writing Data 474 out of 1081 at 06/22/22 13:01:24
Finished Sound Speed Correction 476 out of 1081 at 06/22/22 13:01:25
Finished QAQC 476 out of 1081 at 06/22/22 13:01:26
Finished Beam to True Depth 476 out of 1081 at 06/22/22 13:01:26
Finished Bin Mapping 476 out of 1081 at 06/22/22 13:01:27
Finished Beam to ENU 476 out of 1081 at 06/22/22 13:02:10
Finished QAQC post-coordinate transformation 476 out of 1081 at 06/22/22 13:02:1

/projects/dmcs_1/sw/packages/xroms/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:993: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
/projects/dmcs_1/sw/packages/xroms/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Finished Shear Method 476 out of 1081 at 06/22/22 13:02:16
Finished Writing Data 476 out of 1081 at 06/22/22 13:02:16
Finished Sound Speed Correction 496 out of 1081 at 06/22/22 13:02:31
Finished QAQC 496 out of 1081 at 06/22/22 13:02:32
Finished Beam to True Depth 496 out of 1081 at 06/22/22 13:02:33
Finished Bin Mapping 496 out of 1081 at 06/22/22 13:02:34
Finished Beam to ENU 496 out of 1081 at 06/22/22 13:05:01
Finished QAQC post-coordinate transformation 496 out of 1081 at 06/22/22 13:05:01
Finished Shear Method 496 out of 1081 at 06/22/22 13:05:17
Finished Writing Data 496 out of 1081 at 06/22/22 13:05:17
Finished Sound Speed Correction 500 out of 1081 at 06/22/22 13:05:20
Finished QAQC 500 out of 1081 at 06/22/22 13:05:21
Finished Beam to True Depth 500 out of 1081 at 06/22/22 13:05:21
Finished Bin Mapping 500 out of 1081 at 06/22/22 13:05:21
Finished Beam to ENU 500 out of 1081 at 06/22/22 13:06:17
Finished QAQC post-coordinate transformation 500 out of 1081 at 06/22/22 13:06:1

Finished QAQC 532 out of 1081 at 06/22/22 13:47:30
Finished Beam to True Depth 532 out of 1081 at 06/22/22 13:47:32
Finished Bin Mapping 532 out of 1081 at 06/22/22 13:47:33
Finished Beam to ENU 532 out of 1081 at 06/22/22 13:50:02
Finished QAQC post-coordinate transformation 532 out of 1081 at 06/22/22 13:50:02
Finished Shear Method 532 out of 1081 at 06/22/22 13:50:20
Finished Writing Data 532 out of 1081 at 06/22/22 13:50:20
Finished Sound Speed Correction 534 out of 1081 at 06/22/22 13:50:21
Finished QAQC 534 out of 1081 at 06/22/22 13:50:22
Finished Beam to True Depth 534 out of 1081 at 06/22/22 13:50:24
Finished Bin Mapping 534 out of 1081 at 06/22/22 13:50:25
Finished Beam to ENU 534 out of 1081 at 06/22/22 13:52:57
Finished QAQC post-coordinate transformation 534 out of 1081 at 06/22/22 13:52:57
Finished Shear Method 534 out of 1081 at 06/22/22 13:53:13
Finished Writing Data 534 out of 1081 at 06/22/22 13:53:13
Finished Sound Speed Correction 536 out of 1081 at 06/22/22 13:53:1

Finished Shear Method 563 out of 1081 at 06/22/22 14:38:16
Finished Writing Data 563 out of 1081 at 06/22/22 14:38:16
Finished Sound Speed Correction 564 out of 1081 at 06/22/22 14:38:18
Finished QAQC 564 out of 1081 at 06/22/22 14:38:19
Finished Beam to True Depth 564 out of 1081 at 06/22/22 14:38:21
Finished Bin Mapping 564 out of 1081 at 06/22/22 14:38:22
Finished Beam to ENU 564 out of 1081 at 06/22/22 14:40:56
Finished QAQC post-coordinate transformation 564 out of 1081 at 06/22/22 14:40:56
Finished Shear Method 564 out of 1081 at 06/22/22 14:41:13
Finished Writing Data 564 out of 1081 at 06/22/22 14:41:13
Finished Sound Speed Correction 566 out of 1081 at 06/22/22 14:41:15
Finished QAQC 566 out of 1081 at 06/22/22 14:41:16
Finished Beam to True Depth 566 out of 1081 at 06/22/22 14:41:17
Finished Bin Mapping 566 out of 1081 at 06/22/22 14:41:18
Finished Beam to ENU 566 out of 1081 at 06/22/22 14:44:45
Finished QAQC post-coordinate transformation 566 out of 1081 at 06/22/22 14:44:4

Finished QAQC 597 out of 1081 at 06/22/22 15:35:05
Finished Beam to True Depth 597 out of 1081 at 06/22/22 15:35:07
Finished Bin Mapping 597 out of 1081 at 06/22/22 15:35:08
Finished Beam to ENU 597 out of 1081 at 06/22/22 15:37:50
Finished QAQC post-coordinate transformation 597 out of 1081 at 06/22/22 15:37:50
Finished Shear Method 597 out of 1081 at 06/22/22 15:38:25
Finished Writing Data 597 out of 1081 at 06/22/22 15:38:25
Finished Sound Speed Correction 599 out of 1081 at 06/22/22 15:38:29
Finished QAQC 599 out of 1081 at 06/22/22 15:38:29
Finished Beam to True Depth 599 out of 1081 at 06/22/22 15:38:33
Finished Bin Mapping 599 out of 1081 at 06/22/22 15:38:35
Finished Beam to ENU 599 out of 1081 at 06/22/22 15:41:21
Finished QAQC post-coordinate transformation 599 out of 1081 at 06/22/22 15:41:21
Finished Shear Method 599 out of 1081 at 06/22/22 15:41:39
Finished Writing Data 599 out of 1081 at 06/22/22 15:41:39
Finished Sound Speed Correction 601 out of 1081 at 06/22/22 15:41:4

Finished Shear Method 633 out of 1081 at 06/22/22 16:35:35
Finished Writing Data 633 out of 1081 at 06/22/22 16:35:35
Finished Sound Speed Correction 635 out of 1081 at 06/22/22 16:35:39
Finished QAQC 635 out of 1081 at 06/22/22 16:35:40
Finished Beam to True Depth 635 out of 1081 at 06/22/22 16:35:44
Finished Bin Mapping 635 out of 1081 at 06/22/22 16:35:45
Finished Beam to ENU 635 out of 1081 at 06/22/22 16:38:35
Finished QAQC post-coordinate transformation 635 out of 1081 at 06/22/22 16:38:35
Finished Shear Method 635 out of 1081 at 06/22/22 16:38:50
Finished Writing Data 635 out of 1081 at 06/22/22 16:38:50
Finished Sound Speed Correction 637 out of 1081 at 06/22/22 16:38:51
Finished QAQC 637 out of 1081 at 06/22/22 16:38:52
Finished Beam to True Depth 637 out of 1081 at 06/22/22 16:38:54
Finished Bin Mapping 637 out of 1081 at 06/22/22 16:38:55
Finished Beam to ENU 637 out of 1081 at 06/22/22 16:41:29
Finished QAQC post-coordinate transformation 637 out of 1081 at 06/22/22 16:41:2

## Take a quick peak at results. Actual analysis in different notebook

In [ ]:
path = "/home/jg1200/Data/GliderData/RU29/RU29_2020_AD2CP_NC/Processed/"
files = np.sort(glob.glob(path+'*.csv'))

df = pd.concat(map(pd.read_csv, files))

df

In [ ]:
inversion_depth   = np.arange(5,1030,10)
inversion_lat    = np.empty(len(files))
inversion_lat[:] = np.NaN
inversion_lon    = np.empty(len(files))
inversion_lon[:] = np.NaN
inversion_time    = np.empty(len(files))
inversion_time[:] = np.NaN

u_grid = np.empty((len(inversion_depth),len(files)))
u_grid[:] = np.NaN
v_grid = np.empty((len(inversion_depth),len(files)))
v_grid[:] = np.NaN


## Loop through by file, load in each file
for x in np.arange(0,len(files)):
    
    df = pd.read_csv(files[x])
    u_grid[np.arange(0,len(df.inversion_u.values)),x] = df.inversion_u.values
    v_grid[np.arange(0,len(df.inversion_v.values)),x] = df.inversion_v.values
    inversion_lat[x] = df.latitude.values[0]
    inversion_lon[x] = df.longitude.values[0]
    #inversion_time[x] = pd.to_datetime(df.start_tm.values[0])
      

In [ ]:
plt.figure(figsize=(18,10))
plt.pcolormesh(u_grid,cmap='RdBu',vmin=-0.4,vmax=0.4)
plt.gca().invert_yaxis()
plt.colorbar(label='E-W Velocity [m/s]')
plt.title('RU29 2020')

In [ ]:
plt.figure(figsize=(18,10))
plt.pcolormesh(v_grid,cmap='RdBu',vmin=-0.4,vmax=0.4)
plt.gca().invert_yaxis()
plt.colorbar(label='N-S Velocity [m/s]')
plt.title('RU29 2020')